# Singular Machine Learning Model for Holography
### Grace E. Chesmore and Jeff McMahon - McMahonCosmologyGroup
Here we build a machine learning model for fitting one holography measurement, yielding the LAT mirror adjuster offsets. This ML model takes in 1 holography measurement ("singular") in the form of far-field beams, then converts to aperture fields. The aperture fields and known adjuster offsets are then used as training sets for the ML model (1000 training sets in total).

Prior to using the aperture fields as training sets, the aberrations are subtracted, yielding the pathlength differences due only to the adjuster offsets. The aberration pattern is computed below. 

In [ ]:
import numpy as np
import sys
path_to_holosim_package = "/home/chesmore/Desktop/Code/holosim_paper/package/holosim-ml"
sys.path.append(path_to_holosim_package)
import tele_geo as tg
import ap_field as af
import ap_fitting as afit
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
font_manager.fontManager.addfont(
    "/home/chesmore/.local/share/fonts/times-new-roman.ttf"
)
matplotlib.rcParams["font.family"] = "Times New Roman"
matplotlib.rcParams["font.size"] = 28
plt.rcParams["image.cmap"] = "magma"
plt.style.use("ggplot")
plt.rcParams["axes.unicode_minus"] = False
#%matplotlib inline
import optics_analyze as oa
import sklearn
from sklearn.linear_model import LinearRegression
import pickle

save = 0

rx_x = np.array([0])
rx_z = np.array([0])
el = np.array([0])
az = np.array([0])

shift_C = ["y", oa.sh_z(rx_z[0])]

n_adj_m1 = 5 * 77
n_adj_m2 = 5 * 69

tele_geo = tg.initialize_telescope_geometry()
th = np.linspace(-np.pi / 2 - 0.28, -np.pi / 2 + 0.28, tele_geo.N_scan)
ph = np.linspace(np.pi / 2 - 0.28, np.pi / 2 + 0.28, tele_geo.N_scan)

rx3 = np.array([rx_x[0], 209.09, rx_z[0]])
tele_geo = tg.tele_geo_init(rx3[0], rx3[1], rx3[2], el[0], az[0])
rxmirror_C = af.ray_mirror_pts(rx3, tele_geo, th, ph)
dat_C = afit.take_measurement(
    np.zeros(77 * 5), np.zeros(77 * 5), 0, tele_geo, rxmirror_C
)
dat_C = np.loadtxt(dat_C)
x_C, y_C, meas_C, ampl_C, geo = afit.analyze_holography(
    dat_C, tele_geo, 0, 1, 0, shift_C
)
meas_C = np.where(
    (abs(ampl_C) / np.max(abs(ampl_C))) >= 0.3, meas_C - np.mean(meas_C), 0
)

## Reading in the training sets

In [ ]:
adj_tot1 = []
phases1 = []

iters = 1000

for ii in range(iters):

    if np.mod(ii + 1, 50) == 0:
        print("Reading in measurement " + str(ii + 1) + "/" + str(iters))

    rx3 = np.array([rx_x[0], 209.09, rx_z[0]])
    tele_geo = tg.tele_geo_init(rx3[0], rx3[1], rx3[2], el[0], az[0])
    dat_C = np.loadtxt(
        "/data/chesmore/sim_out/rx000/rx_" + str(rx3) + "_" + str(ii + 1) + ".txt"
    )
    x_C, y_C, phase_C, ampl_C, geo = afit.analyze_holography(
        dat_C, tele_geo, 0, 1, 0, shift_C
    )

    phase_C = np.where(
        (abs(ampl_C) / np.max(abs(ampl_C))) >= 0.3, phase_C - np.mean(phase_C), 0
    )
    phase_C -= meas_C

    adj_m1 = np.loadtxt(
        "/data/chesmore/sim_out/rx000/adj_offsets_m1_" + str(ii + 1) + ".txt"
    )[
        0:n_adj_m1
    ]  # mm

    adj_m2 = np.loadtxt(
        "/data/chesmore/sim_out/rx000/adj_offsets_m2_" + str(ii + 1) + ".txt"
    )[
        0:n_adj_m2
    ]  # mm

    phases1.append((phase_C))
    adj_tot1.append(np.concatenate((adj_m1, adj_m2)))

    if ii == 0:
        plt.figure(figsize=(5, 3.5))
        plt.title("Singular holography measurements", fontsize=20, x=0.44, y=1.15)
        plt.scatter(x_C, y_C, c=1e6 * phase_C / tele_geo.k, vmin=-200, vmax=200)
        plt.axis("equal")
        plt.xlabel("x [m]")
        plt.ylabel("y [m]")
        plt.colorbar(label=r"$\mu m$")
        plt.show()

## Training and saving the ML model

In [ ]:
####### Singular Model ################
n_samples = np.shape(phases1)[0]
n_features = np.shape(phases1)[1]

# Define training datasets
X = phases1[0 : int(iters - 1)]
y_output = adj_tot1[0 : int(iters - 1)]

# Train the linear regression ML model
model1 = LinearRegression()  # Define ML algorithm
model1.fit(X, y_output)  # Train model with training datasets

# Introduce instances where we do not know the answer (holography measurement)
Xnew = np.reshape(phases1[int(iters - 1)], (1, n_features))
# Make a prediction
ynew = model1.predict(Xnew)

# Save the model
filename_si = "model_singular1.sav"
pickle.dump(model1, open(filename_si, "wb"))

In [ ]:
plt.plot(ynew[0])
plt.plot(adj_tot1[int(iters - 1)])
plt.xlim(500, 600)